# Instructions:-

All questions are compulsory.

In each of the questions you have to automate the process. You do not have to click on any button, click any clickable element, enter keywords in search boxes manually. Each process has to be performed via coding.

Q1 and Q2 are connected questions i.e. after attempting Q1 proceed to Q2. Do not write whole code from beginning for Q2.

You may use any web scraping library and tools.

The question can be attempted in various ways; the correctness of question depends on the output.

If you encounter any Null values during scraping, you may replace it by hyphen.

In [13]:
#importing all the required libraries-
import selenium
from selenium import webdriver
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from datetime import datetime
from selenium.common.exceptions import ElementNotInteractableException
print("All the libraries are successfully loaded")

All the libraries are successfully loaded


# Question-1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [16]:
#loading the drivers and URL

driver=webdriver.Chrome('chromedriver.exe')
driver.maximize_window()   #using this function for maximizing the window
driver.get('http://amazon.in/')
driver.implicitly_wait(5) 

#entering the details in the search column using user input-
product_vertical=input("Enter product vertical that needs to be searched:-")
driver.find_element_by_id("twotabsearchtextbox").send_keys(product_vertical)
driver.find_element_by_id("nav-search-submit-button").click()

Enter product vertical that needs to be searched:-guitar


# Question2.- In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [19]:
#creating a function to get the details of all products from the first three pages-
urls=[]
for z in range(3):
    for i in driver.find_elements_by_xpath("//div[@class='a-section a-spacing-medium']//h2"):
        urls.append(i.find_element_by_xpath(".//a").get_attribute('href'))
    try:
        next_page=driver.find_element_by_xpath("//div[@class='a-text-center']/ul/li[@class='a-last']/a").get_attribute('href')
        driver.get(next_page)
        driver.refresh()   
    except:
        break
        
        
#creating empty list to store our data-
brand_name=[]
name_of_the_product=[]
rating=[]
no_of_ratings=[]
price=[]
return_exchange=[]
expected_delivery=[]
availability=[]
other_details=[]
product_url=[]

#talking all URL's one by one separtely-
for i in urls:
    driver.get(i)
    product_url.append(i)
    driver.implicitly_wait(5)
    

    
#scrping the brand name
    try: brand_name.append(driver.find_element_by_id("productTitle").text.split(' ')[0]) 
    except: brand_name.append('-')
    driver.implicitly_wait(5)
    
    
    
#scraping the name of product
    try: 
        string=''
        for i in driver.find_element_by_id("productTitle").text.split(' ')[1:]:
            string=string+' '+i
        name_of_the_product.append(string)
    except: name_of_the_product.append('-')
    driver.implicitly_wait(5)

    
#scraping the rating of the product
    try: rating.append(driver.find_element_by_id("acrPopover").get_attribute('title')) 
    except: rating.append('-')
    driver.implicitly_wait(5)
    
    
    
#scraping the number of ratings
    try: no_of_ratings.append(driver.find_element_by_id("acrCustomerReviewText").text) 
    except: no_of_ratings.append('-')
    driver.implicitly_wait(5)
    
    
    
#scraping the price of the product
    try: 
        try: price.append(driver.find_element_by_id("priceblock_saleprice").text) 
        except:
            try: price.append(driver.find_element_by_id("priceblock_dealprice").text)
            except: price.append(driver.find_element_by_id("priceblock_ourprice").text)
    except: price.append('-')
    driver.implicitly_wait(5)
    
    
    
#scraping the details of return/exchnage of the product
    try: 
        return_row=driver.find_element_by_xpath("//div[@class='a-row icon-farm-wrapper']").find_elements_by_xpath(".//div")
        found=False
        for i in return_row:
            if(i.get_attribute('data-name')=='RETURNS_POLICY'):
                found=True
                return_exchange.append(i.find_element_by_xpath(".//span[1]/div[2]/a").text)
        if(found==False):
            return_exchange.append('-')
    except: return_exchange.append('-')
    driver.implicitly_wait(5)
    
    
    
#scraping the expected delivery date
    try: expected_delivery.append(driver.find_element_by_id("ddmDeliveryMessage").find_element_by_xpath(".//b").text)
    except: expected_delivery.append('-')
    driver.implicitly_wait(5)
    
    
    
#scraping the availablity of the product
    try: 
        try: availability.append(driver.find_element_by_id("availability").find_element_by_xpath(".//span").text)
        except: availability.append(driver.find_element_by_id("deal_availability").find_element_by_xpath(".//div/span").text)
    except: availability.append('-')
    driver.implicitly_wait(5)
    
    
    
#scraping other details
    try:
        details=[i.text.replace('\n','---') for i in driver.find_element_by_id("productDetails_techSpec_section_1").find_elements_by_xpath(".//tbody")] 
        other_details.append(details[0])
    except: other_details.append('-')
    driver.implicitly_wait(5)

In [21]:
#creating a dataframe of our scapped data-
Guitar=pd.DataFrame({'Brand Name':brand_name,'Name of Product':name_of_the_product,'Rating':rating,'No. of Ratings':no_of_ratings,'Price':price,
             'Return/Exchange':return_exchange,'Expected Delivery':expected_delivery,'Availability':availability,'Other details':other_details,
             'Product URL':product_url})

In [22]:
Guitar

,Brand Name,Name of Product,Rating,No. of Ratings,Price,Return/Exchange,Expected Delivery,Availability,Other details,Product URL
0,Juârez,"Acoustic Guitar, 38 Inch Cutaway, 038C with B...",3.9 out of 5 stars,"9,714 ratings","₹ 2,499.00",7 Days Replacement,"Friday, June 11",In stock.,Product Dimensions 98 x 40 x 10 cm; 2.06 Kilog...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Kadence,Slowhand Premium Jumbo Size Semi Acoustic Gui...,4.3 out of 5 stars,128 ratings,"₹ 10,999.00",7 Days Replacement,"Saturday, June 12",In stock.,Brand Kadence---Manufacturer Kadence---Model S...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Juarez,"JRZ38C Acoustic Guitar, 38 Inch Cutaway with ...",4.0 out of 5 stars,"3,323 ratings","₹ 2,499.00",7 Days Replacement,"Friday, June 11",In stock.,Product Dimensions 98 x 40 x 10 cm; 2.1 Kilogr...,https://www.amazon.in/gp/slredirect/picassoRed...
3,Juarez,"Acoustic Guitar, 38 Inch Cutaway, 38CMB with ...",3.6 out of 5 stars,160 ratings,"₹ 2,499.00",7 Days Replacement,"Friday, June 11",In stock.,Product Dimensions 7.6 x 12.7 x 81.3 cm; 2.62 ...,https://www.amazon.in/gp/slredirect/picassoRed...
4,Juârez,"Acoustic Guitar, 38 Inch Cutaway, 038C with B...",3.9 out of 5 stars,"9,714 ratings","₹ 2,499.00",7 Days Replacement,"Friday, June 11",In stock.,Product Dimensions 98 x 40 x 10 cm; 2.06 Kilog...,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...,...,...,...
179,SG,Musical - 12MM Electric Guitar Gig Bag,-,-,₹ 399.00,7 Days Replacement,"Monday, June 14",Only 1 left in stock.,Brand SG Musical---Manufacturer SG Musical---M...,https://www.amazon.in/SG-Musical-12MM-Electric...
180,JUAREZ,"Arpeggio Guitar JRA39SR-BL-K, blue",-,-,"₹ 6,729.00",7 Days Replacement,"Friday, June 11",In stock.,Product Dimensions 98 x 40 x 10 cm; 2.3 Kilogr...,https://www.amazon.in/gp/slredirect/picassoRed...
181,SG,Musical - 12MM Electric Guitar Gig Bag,-,-,₹ 399.00,7 Days Replacement,"Monday, June 14",Only 1 left in stock.,Brand SG Musical---Manufacturer SG Musical---M...,https://www.amazon.in/gp/slredirect/picassoRed...
182,HYMET,"4-String Acoustic Guitar Learning Kids Toy, B...",2.0 out of 5 stars,1 rating,₹ 789.00,7 Days Returnable,"Saturday, June 12",In stock.,Number of Game Players 1---Number of Puzzle Pi...,https://www.amazon.in/gp/slredirect/picassoRed...


In [23]:
#saving the dataframe in csv file-
Guitar.to_csv('guitar_products.csv')

# Question3.- Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [46]:
#loading the URL & maxizing window.
driver=webdriver.Chrome('chromedriver.exe')
driver.maximize_window()
url=('https://images.google.com/')
driver.get(url)
driver.implicitly_wait(3)

In [55]:
#define a function which accepts image as an argument-
def get_urls(keywords):
        driver.get(url)
        
        #entering the details in the search column using xpath function and click search button
        search_option=driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
        search_option.send_keys(keywords)
        btn=driver.find_element_by_xpath("//button[@class='Tg7LZd']")
        btn.click()
        driver.implicitly_wait(5)
        
        
#loading pages for scraping images from them-
        for z in range(10):
            driver.find_element_by_xpath("//a[@class='wXeWr islib nfEiy mM5pbd']").send_keys(Keys.PAGE_DOWN)
        time.sleep(5)
        pics=[k for k in driver.find_elements_by_xpath("//a[@class='wXeWr islib nfEiy mM5pbd']")[:100]]
        urls=[]
        print(len(pics))  
        try:
            if(len(pics)==100):
                for i in pics:
                    try:
                        i.click()
                        urls.append(i.get_attribute('href'))
                    except:
                        urls.append('Not Available')
                return urls
            else:
                print('Image Not Available')
        except:
            pass

In [56]:
#scraping the url's of all images

fruits=get_urls('Fruits')
cars=get_urls('Cars')
machine_learning=get_urls('Machine Learning')

100
100
100


In [57]:
#creating a dataframe for our data-

Img_url=pd.DataFrame({})
Img_url['Fruits']=fruits
Img_url['Cars']=cars
Img_url['Machine-Learning']=machine_learning
Img_url

,Fruits,Cars,Machine-Learning
0,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
1,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
2,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
3,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
4,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
...,...,...,...
95,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
96,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
97,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
98,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=http%3A%2...,https://www.google.com/imgres?imgurl=https%3A%...


# Question 4 - Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [58]:
#loading the drivers, URL and maximizing window.
driver=webdriver.Chrome('Chromedriver.exe')
driver.maximize_window()
driver.get('https://www.flipkart.com/')
driver.implicitly_wait(2)

In [59]:
#searching the details in the search column in source browser-
product=input('Enter the product vertival that needs to be searched : ')
driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]').click()
driver.implicitly_wait(2)
driver.find_element_by_xpath('//input[@title="Search for products, brands and more"]').send_keys(str(product))
driver.find_element_by_xpath('//button[@type="submit"]').click()
driver.implicitly_wait(2)

Enter the product vertival that needs to be searched : smartphone


In [60]:
#creating an empty list to store the scraped data -

product_urls=[]
brand_name=[]
smartphone_name=[]
smartphone_color=[]
RAM=[]
ROM=[]
primary_camera=[]
secondary_camera=[]
display_size=[]
display_resolution=[]
processor=[]
processor_core=[]
battery_capacity=[]
price=[]

In [62]:
#defining a function-
url=driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')

for i in url:
    product_urls.append(i.get_attribute('href'))

In [63]:
#scraping product URL    
for i in product_urls:
    driver.get(i)
    driver.implicitly_wait(4)
    driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()
    
#scraping brand name-
    try:
        title=driver.find_element_by_xpath('//span[@class="B_NuCI"]').text.split()
        brand_name.append(title[0])
    except:
        brand_name.append('-')
        

#scraping name-
    try:
        smartphone_name.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[3]/td[2]').text)
    except:
        smartphone_name.append('-')
        

#scraping colour-
    try:
        smartphone_color.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[4]/td[2]').text)
    except:
        smartphone_color.append('-')
        
        
        
#scraping RAM-
    try:
        RAM.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[2]/td[2]').text)
    except:
        RAM.append('-')
        

#scraping ROM-
    try:
        ROM.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[1]/td[2]').text)
    except:
        ROM.append('-')
        
#scraping primary camera details-
    try:
        primary_camera.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[2]/td[2]').text)
    except:
        primary_camera.append('-')
        
        
#scraping secondary camera details-
    try:
        secondary_camera.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[5]/td[2]').text)
    except:
        secondary_camera.append('-')
        
        
#scraping display size-
    try:
        display_size.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[1]/td[2]').text)
    except:
        display_size.append('-')
        
        
#scraping screen resolution-
    try:
        display_resolution.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[2]/td[2]').text)
    except:
        display_resolution.append('-')
        
        
#scraping processor-
    try:
        processor.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[2]/td[2]').text)
    except:
        processor.append('-')
        
        
#scraping  processor core-
    try:
        processor_core.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[3]/td[2]').text)
    except:
        processor_core.append('-')
        
        
#scraping  battery capacity-
    try:
        battery_capacity.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[10]/table/tbody/tr[1]/td[2]').text)
    except:
        battery_capacity.append('-')
        
        
#scraping  price-
    try:
        price.append(driver.find_element_by_xpath('//div[@class="dyC4hf"]/div/div/div').text)
    except:
        price.append('-')        


In [64]:
#creating a dataframe of our scraped data-
SMARTPHONE=pd.DataFrame({})
SMARTPHONE['Brand-Name']=brand_name
SMARTPHONE['Smartphone-Name']=smartphone_name
SMARTPHONE['Colour']=smartphone_color
SMARTPHONE['RAM']=RAM
SMARTPHONE['Storage(ROM)']=ROM
SMARTPHONE['Primray-Camera']=primary_camera
SMARTPHONE['Secondary-Camera']=secondary_camera
SMARTPHONE['Display-Size']=display_size
SMARTPHONE['Display-Resolution']=display_resolution
SMARTPHONE['Processor']=processor
SMARTPHONE['Processor-Cores']=processor_core
SMARTPHONE['Battery-Capacity']=battery_capacity
SMARTPHONE['Price']=price
SMARTPHONE['Product-URL']=product_urls

In [65]:
SMARTPHONE

,Brand-Name,Smartphone-Name,Colour,RAM,Storage(ROM),Primray-Camera,Secondary-Camera,Display-Size,Display-Resolution,Processor,Processor-Cores,Battery-Capacity,Price,Product-URL
0,POCO,M3,Cool Blue,6 GB,64 GB,48MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 662,Octa Core,77.3 mm,"₹10,499",https://www.flipkart.com/poco-m3-cool-blue-64-...
1,REDMI,9i,Midnight Black,4 GB,64 GB,13MP Rear Camera,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,MediaTek Helio G25,Octa Core,5000 mAh,"₹7,999",https://www.flipkart.com/redmi-9i-midnight-bla...
2,Infinix,Hot 10 Play,7° Purple,4 GB,64 GB,13MP + Depth Sensor,8MP Front Camera,17.32 cm (6.82 inch),1640 x 720 Pixels,MediaTek Helio G35,Octa Core,6000 mAh,"₹8,499",https://www.flipkart.com/infinix-hot-10-play-7...
3,Infinix,Hot 10 Play,Morandi Green,4 GB,64 GB,13MP + Depth Sensor,8MP Front Camera,17.32 cm (6.82 inch),1640 x 720 Pixels,MediaTek Helio G35,Octa Core,6000 mAh,"₹8,499",https://www.flipkart.com/infinix-hot-10-play-m...
4,REDMI,Note 9,Pebble Grey,4 GB,64 GB,48MP + 8MP + 2MP + 2MP,Yes,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh,"₹10,999",https://www.flipkart.com/redmi-note-9-pebble-g...
5,Infinix,Hot 10 Play,Aegean Blue,4 GB,64 GB,13MP + Depth Sensor,8MP Front Camera,17.32 cm (6.82 inch),1640 x 720 Pixels,MediaTek Helio G35,Octa Core,6000 mAh,"₹8,499",https://www.flipkart.com/infinix-hot-10-play-a...
6,realme,C20,Cool Blue,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,MediaTek Helio G35,Octa Core,76.4 mm,"₹6,799",https://www.flipkart.com/realme-c20-cool-blue-...
7,realme,C20,Cool Grey,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,MediaTek Helio G35,Octa Core,76.4 mm,"₹6,799",https://www.flipkart.com/realme-c20-cool-grey-...
8,realme,Narzo 30A,Laser Blue,3 GB,32 GB,13MP + 2MP,Yes,16.54 cm (6.51 inch),1600 x 720 Pixels,MediaTek Helio G85,Octa Core,6000 mAh,"₹7,999",https://www.flipkart.com/realme-narzo-30a-lase...
9,realme,Narzo 30A,Laser Black,3 GB,32 GB,13MP + 2MP,Yes,16.54 cm (6.51 inch),1600 x 720 Pixels,MediaTek Helio G85,Octa Core,6000 mAh,"₹7,999",https://www.flipkart.com/realme-narzo-30a-lase...


In [66]:
#Saving the dataframe in csv file-
SMARTPHONE.to_csv('online_flipkart_smartphone_collection.csv')

# Question 5 - Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [67]:
#loading the drivers,URL & maximizig the window-
driver=webdriver.Chrome('chromedriver.exe')
driver.maximize_window()
driver.get('https://www.google.com/maps/')
driver.implicitly_wait(3)

In [71]:
#defining function-
def main(): 
    Place = input('Enter Name Of Place to search for->')
    
    #Searching by xpath function
    search = driver.find_element_by_xpath("//div[@id='gs_lc50']/input")
    search.send_keys(Place)
    
    #clicking search_button
    btn = driver.find_element_by_xpath("//button[@id='searchbox-searchbutton']")
    btn.click()
    
    print('Location details-')
    time.sleep(10)
    
    #Getting current_URL
    curr_url = driver.current_url
    
    #Scrapping results from the entered city latitude and Longitude from google map
    Latitude=driver.current_url.split('/@')[1].split(',')[0]
    Longitude=driver.current_url.split('/@')[1].split(',')[1].split(',')[0]

    print('Latitide of ',Place,' =',Latitude)
    print('Longitude of ',Place,' =',Longitude)
    

#main function calling-
if __name__ == "__main__":
    main()

Enter Name Of Place to search for->Pune
Location details-
Latitide of  Pune  = 18.5248904
Longitude of  Pune  = 73.7228781


# Question 6 - Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [72]:
#loading the drivers,URL & maximizing the window-
driver=webdriver.Chrome('chromedriver.exe')
driver.maximize_window()
driver.get('https://trak.in/')
driver.implicitly_wait(3)

In [73]:
#searching  column using xpath functionality
btn = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(btn)

In [77]:
#for july finding elements by xpath-
driver.find_element_by_xpath("//div[@id='tablepress-48_length']/label/select/option[4]").click()  
#for august finding elements by xpath-
driver.find_element_by_xpath("//div[@id='tablepress-49_length']/label/select/option[4]").click() 
#for september finding  elements by xpath-
driver.find_element_by_xpath("//div[@id='tablepress-50_length']/label/select/option[4]").click()  
#downloading page source
html = driver.page_source


#reading tables using pandas
df = pd.read_html(html)

#selecting required dataframe for each months-
frames = [df[10],df[12],df[11]]

#concatenating dataframes of each month and printing the result-
Funding_Deals = pd.concat(frames,ignore_index=True,join="inner")

In [78]:
Funding_Deals

,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD)
0,1,30/11/2020,Codingal,EduTech,Online coding platform,Bengaluru,"Rebright Partners, Java Capital, and GSF Accel...",Seed,560000
1,2,30/11/2020,CRED,Financial Services,Credit Card Behavior Services,Bengaluru,DST Global,Series C,80000000
2,3,29/11/2020,Bira 91,Food and Beverage,Craft Beer,New Delhi,"Sofina, Sequoia Capital, Sixth Sense Ventures,...",Bridge Funding,3000000
3,4,27/11/2020,Chumbak,Lifestyle,Gifting Startup,Bengaluru,Gaja Capital Fund (GCFII-A),Pre-series E,1001047.92
4,5,25/11/2020,Cityflo,Transportation,App-based bus commute startup,Mumbai,Lightbox Ventures,Series A,7700000
5,6,25/11/2020,Shipsy,SaaS,Software Automation Of Logistics,Gurgaon,"Sequoia Capital India’s Surge, Info Edge",Series A,6003806
6,7,25/11/2020,Unacademy,Edutech,Online Learning Platform,Bengaluru,"Tiger Global Management, Dragoneer Investment ...",Venture,75000000-100000000
7,8,24/11/2020,Cashfree,FinTech,Payments and banking technology,Bengaluru,"Matrix Partners India, SAIF Partners, Ward Fer...",Series B,35300000
8,9,30/11/2020,Credgenics,Fintech,Critical NPA reduction for Fintech lenders and...,Gurgaon,"Venture Capital firm, Accel Partners and DMI F...",Pre-Series A,3500000
9,10,23/11/2020,Country Delight,Dairy Tech,Milk delivery startup,Gurugram,Elevation Capital,Series C,25000000


# Question 7 - Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [79]:
#loading the drivers,URL & maximizing the window-
driver=webdriver.Chrome('Chromedriver.exe')
driver.get('https://www.digit.in/')
driver.maximize_window()
driver.implicitly_wait(5)

In [80]:
#Searching for the link of best gaming laptop-
btn = driver.find_element_by_xpath("//div[@class='listing_container']/ul/li[9]/a").get_attribute('href')
driver.get(btn)

In [83]:
#downloading page source
html = driver.page_source

#reading tables using pandas-
df = pd.read_html(html)

#Scrapping specification_tables-
import numpy as np
num = np.arange(1,11)
laptop= []
for i in num:
    df1 = df[1].drop('SPECIFICATION.1', axis = 1)
    df2 = df1.drop('SPECIFICATION', axis = 1)
    ds = df2.transpose()    
    laptop.append(ds)

In [85]:
#Joining specification_tables-
LAPTOP = pd.concat(laptop,ignore_index=True,join="inner")
LAPTOP['Product_Name']=df[0]['Product Name']
LAPTOP['Seller']=df[0]['Seller']
LAPTOP['Price']=df[0]['Price']

In [88]:
#Renaming final Table
LAPTOP = LAPTOP.rename(columns={0 : "OS", 1: "Display",2: "Processor",3: "Memory",4: "Weight",5: "Dimension",6: "Graphics Processor"})

In [89]:
LAPTOP

,OS,Display,Processor,Memory,Weight,Dimension,Graphics Processor,Product_Name,Seller,Price
0,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,Alienware Area 51M R2,NaN,NaN
1,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,Alienware m15 R3,NaN,₹341990
2,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,ASUS ROG Strix Scar 15,NaN,NaN
3,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,Asus ROG Zephyrus G14,amazon,₹164990
4,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,Lenovo Legion 5i,NaN,₹71990
5,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,ASUS ROG ZEPHYRUS DUO 15,amazon,₹199990
6,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,Acer Aspire 7 gaming,NaN,₹56990
7,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,NaN,NaN,NaN
8,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,NaN,NaN,NaN
9,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,NaN,NaN,NaN


# COMPLETED-->>